# TeraStore

**TeraStore** is the IDS implementation by teralab of an AppStore based on the TeraLab Marketplace. This notebook will cover in length the features and implementation of the TerStore.


## How it came to be

**TeraStore** is mostly inspired by the EUH4D implementation that is, itself based on Dataspace Connector implementation, the most well-documented connector implementation available as of today (as far as we know).
By combining the DataSpace Connector implementation with the documentation found on JIVE (in particular the IDS communication guide and the IDS handshake documents), we created the TeraStore, an *incomplete* AppStore implementation attempt with the following features :

* combines Apps from two different sources:
  * Trusted Network Apps: as the name suggests these are Apps created and managed by IDS connectors from the inside the trusted network.
  * TeraLab Marketplace Apps: are Apps originally on the TeraLab marketplace that the AppStore fetches and add to the Catalog and by doing so declares itself as author for them. This works since an AppStore can be viewed as a particular connector.
* ~~provides a GUI allowing to browse through the catalog~~
* provides an infrastructure endpoint supporting the following mesages: 
  * DescriptionRequestMessage : to request the self-description JSON-LD of the AppStore
  * AppRegistrationRequestMessage : to create an App using metadata
  * AppUploadingMessage: to upload App deployment Spec (keep reading to understand what this refers to)
  * QueryMessage: fetches the catalog
  * RequestArtifactMessage : fetches the actuall App deployment spec 
<!--* provides an OpenAPI access to get familiar with the way things work.-->

Note: A more complete list is available [here](http://htmlpreview.github.io/?https://github.com/IndustrialDataSpace/InformationModel/blob/feature/message_taxonomy_description/model/communication/Message_Description.htm)

The following features are not yet available but are planned in the road map: 

* support for SPARQL 
* support for other IDS messages (such as notification messages)


## What is an App ?

Based on the concept of a TeraLab Marketplace App, we decided that an App is essentially a docker specification. It has two groups of information : 1) the metadata, helping with defining and classifying Apps and 2) the deployment, containing necessary information to deploy a docker container in a configurable way (it allows providing volumes and specifying environment variables). Both the metadata and deployment are JSON document with particular formatting that we are going detail now.

The Metadata has the following fields :
* __name__: the name of the App, mandatory.
* __introduction__: a less than 200 characters string introducing the App, optional.
* __description__: a long (no limit on the number of characters), HTML-formatted (graphical) string describing fully the App. This field can be viewed as the Apps user manual. This field s optional.
* __keywords__: Tags associated to the App, optional.

The deployment has the following fields :
* __image__: docker image to pull / use when creating the container, mandatory.
* __ports__: an array of portObjects describing the ports to pay attention to, optional. This is how a JSON portObject is structured :
  * __name__: usually the Application level protocol used (ex: http), mandatory
  * __containerPort__: the target port (ex: 80), mandatory
* __settings__: an array of JSON settingsObjects describing the environment variables to specify. This is how a JSON settingsObject is structured :
  * __name__: name of the env variable, mandatory.
  * __type__: expected type (string, number, date, url, ...), optional, default is string.
  * __required__: specifies if the env variable is mandatory for the container to run successfully, optional, default is false 
* __volumes__: an array of JSON volumeObjects describing docker volumes expected, optional. This can allow the same container with an algorithm as main process for example to work on different data simply by providing a different volume (containing a different dataset but in the expected format, e.g. CSV files). This is how a JSON volumeObject is structured:
  * __name__: name of the volume (usually what kind of data expected there), mandatory.
  * __mountPoint__: where the specified volume should be mounted, mandatory.

The following figure shows how a typical can App deployment specification can translate into a docker deployment when provided with the right context (settings and volumes) :

![Alt text](images/app_deploy_spec.PNG)
 

## Architecture

This is the architecture of AppStore as well as a typical exchange : 

![Alt text](images/archi.PNG)




## Demo

The demo involves : 
* a DAPS : the one used belongs to SQS, contact [toto](mailto:) for more information
* a DataBase: to record Apps Metadata and Artifacts 
* the TeraLab Marketplace: to where to retrieve addiionnal Apps 
* the notebook : we are currently, will simulate two connectors, namely an App provider and an AppConsumer.

The scenario is the follwing :

* Creation of two participants and a matching connector identities on the DAPS.
  * Ask for the maintainers of the DAPS to create participants and connector identities.
  * Edit the [provider](specs/provider.json) and the [consumer](specs/consumer.json) files containing a self description for each of the simulated connectors of the demo
  * Open up the [idsComm.py](./idsComm.py) and replace the "PROVIDER_DAT" and "CONSUMER_DAT" by the right DAT provided by the DAPS. 

* Adapt the configuration of the TeraStore
  * open the appstore/settings/settings.yaml file and replace the parameters with your own
  * clean up the current context by typing:
    * <pre><code>docker-compose down --remove-orphans --volumes</code></pre>
  * start the server using:
    * <pre><code>docker-compose up</code></pre>

* Through the infrastructure endpoint (find logs of last exchanges in the "logs" folder)
  * Get the self-description of the AppStore 
  * Get the Catalog (collection of metadata)
  * Get an Actual Apps as an artifact (deployment spec) 
  * Create a new App
  * Check that the new App appears in the Catalog

**/!\ THe first and second points are very manual, as such they cannot be automated in the notebook, but do not neglect them if you wish to adapt TeraStore to your own needs. Therefore only the third point shall be covered in this notebook.**

Before we get down to business, let us first setup the environment correctly.


In [1]:
!pip3 install requests-toolbelt

from idsComm import getSelfDescription, getCatalog, createApp, artifactRequestMessage
import json

# Check the logs fold to read the transactions.
LOG_EXCHANGES = True

### Self description

The next cell explains how to get the self description :

In [2]:

# Operation done by the consumer connector
header, payload = getSelfDescription("consumer", LOG_EXCHANGES)

print(json.dumps(payload, indent=4))

{
    "@context": "http://w3id.org/idsa/contexts/context.json",
    "@id": "https://terastore.tl.teralab-datascience.fr",
    "ids:correlationMessage": "https://terastore.tl.teralab-datascience.fr",
    "@type": "ids:AppStore",
    "ids:description": {
        "@value": "An implementation effort by TeraLab",
        "@language": "en"
    },
    "ids:curator": "https://www.teralab-datascience.fr",
    "ids:maintainer": "https://www.teralab-datascience.fr",
    "ids:inboundModelVersion": [
        "4.0.0"
    ],
    "ids:outboundModelVersion": "4.0.0",
    "ids:title": {
        "@value": "TeraStore",
        "@language": "en"
    },
    "ldp:contains": {
        "@id": "Catalog"
    },
    "ids:catalog": {
        "@id": "https://terastore.tl.teralab-datascience.fr/catalog",
        "@type": [
            "ids:Catalog",
            "ldp:BasicContainer"
        ],
        "ids:offer": [
            {
                "@id": "https://terastore.tl.teralab-datascience.fr/apps/5fb633ea4f5aa70


### Catalog

The next cell explains how to get the Catalog :

In [3]:

# Operation done by the consumer connector
header, payload = getCatalog("consumer", LOG_EXCHANGES)

print(json.dumps(payload, indent=4))

[
    {
        "@id": "https://terastore.tl.teralab-datascience.fr/apps/5fb633ea4f5aa7013ddae944",
        "@type": [
            "ids:AppResource"
        ],
        "ids:description": "Test full description",
        "ids:title": "algorun",
        "ids:comment": "A simple SKLEARN \"box\" ready to run your python codes.",
        "ids:keywords": [
            "from:marketplace"
        ],
        "ids:artifact": [
            {
                "@id": "https://terastore.tl.teralab-datascience.fr/apps/5fb633ea4f5aa7013ddae944"
            }
        ]
    },
    {
        "@id": "https://terastore.tl.teralab-datascience.fr/apps/6021209f68a66200295c871e",
        "@type": [
            "ids:AppResource"
        ],
        "ids:description": "<p>Full description&nbsp;</p>",
        "ids:title": "newApp",
        "ids:comment": "introduction to my new app",
        "ids:keywords": [
            "from:marketplace"
        ],
        "ids:artifact": [
            {
                "@id": "h

**/!\ You can differentiate Apps coming from the TeraLab Marketplace from the "Trusted Network Apps" byt the "from:marketplace" keyword. The apps having this keyword are most probably (not certainly since anyone can affect any keyword to their resources) from the TeraLab Marketplace.**

### Requesting Artifact 

The next cell explains how to get the Artifact for an App (deployment spec) :

In [4]:
header, payload = artifactRequestMessage("https://terastore.tl.teralab-datascience.fr/apps/5fb633ea4f5aa7013ddae944", LOG_EXCHANGES)

print(json.dumps(payload, indent=4))

{
    "image": "algofab2018/algorun",
    "ports": [
        {
            "_id": "60379231d55cff049773e7df",
            "name": "http"
        }
    ],
    "settings": [],
    "volumes": [
        {
            "_id": "60379231d55cff049773e7e0",
            "name": "my-volume"
        }
    ]
}


### Creating App


Now let us create an App

In [5]:

# Operation done by the provider connector

app = {
    "metadata": {
        "name": "Example nginx",
        "keywords": [
            "nginx", "proxy"
        ],
        "introduction": "Simple App",
        "description": "<p>TeraStore example App.</p>"
    },
    "spec": {
        "image": "nginx",
        "ports": [
            {
                "name": "http",
                "port": 80
            }
        ]
    }
}

c_header, c_payload, u_header, u_payload = createApp(app, LOG_EXCHANGES)

**After creating the App, this was the result**

In [6]:
print(json.dumps(c_payload, indent=4))

{
    "@id": "https://terastore.tl.teralab-datascience.fr/apps/6037944de6335404a22e5bb4",
    "@type": [
        "ids:AppResource"
    ],
    "ids:description": "<p>TeraStore example App.</p>",
    "ids:title": "Example nginx",
    "ids:comment": "Simple App",
    "ids:keyword": [
        "nginx",
        "proxy"
    ],
    "ids:artifact": [
        {
            "@id": "https://terastore.tl.teralab-datascience.fr/apps/6037944de6335404a22e5bb4"
        }
    ]
}


**After uploading deployment spec, this was the result**

In [7]:
print(json.dumps(u_payload, indent=4))

{
    "ports": [
        {
            "_id": "6037944ee6335404a22e5bb6",
            "name": "http",
            "port": 80
        }
    ],
    "settings": [],
    "volumes": [],
    "image": "nginx"
}



### Checking the Catalog again


Finally, let us fetch the catalog and see if the App was created as expected (should be the last item on the catalog).

In [8]:
# Operation done by the consumer connector
header, payload = getCatalog("consumer", LOG_EXCHANGES)

print(json.dumps(payload, indent=4))

[
    {
        "@id": "https://terastore.tl.teralab-datascience.fr/apps/5fb633ea4f5aa7013ddae944",
        "@type": [
            "ids:AppResource"
        ],
        "ids:description": "Test full description",
        "ids:title": "algorun",
        "ids:comment": "A simple SKLEARN \"box\" ready to run your python codes.",
        "ids:keywords": [
            "from:marketplace"
        ],
        "ids:artifact": [
            {
                "@id": "https://terastore.tl.teralab-datascience.fr/apps/5fb633ea4f5aa7013ddae944"
            }
        ]
    },
    {
        "@id": "https://terastore.tl.teralab-datascience.fr/apps/6021209f68a66200295c871e",
        "@type": [
            "ids:AppResource"
        ],
        "ids:description": "<p>Full description&nbsp;</p>",
        "ids:title": "newApp",
        "ids:comment": "introduction to my new app",
        "ids:keywords": [
            "from:marketplace"
        ],
        "ids:artifact": [
            {
                "@id": "h